# This is the project in Optimization 1

In [ ]:
from functions import *
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import default_rng
from sklearn.metrics.pairwise import pairwise_kernels


In [ ]:
w = np.array([1.,1.])
b = 1.
n_A = 100
n_B = 80
margin = 5.e-1
listA,listB = TestLinear(w,b,n_A,n_B,margin)
[plt.scatter(x[0],x[1],color="r") for x in listA]
[plt.scatter(x[0],x[1],color="b") for x in listB]
plt.show()


In [ ]:
x= np.array(listA)
y = np.array([1]*n_A )

G = pairwise_kernels(x, metric = kernal_gaussian, sigma=1)

print(G)

### Projected gradient descent algorithm

\begin{align*}
    \alpha^{(k+1)} &= \alpha^{(k)}+ d^{(k)}\\
    \text{where}&\\
    d^{(k)} & = \pi _{\Omega} \left(\alpha^{(k)}-\tau_k \nabla f(\alpha^{(k)}) \right) - \alpha^{(k)}
\end{align*}

\begin{align*}
    f(\alpha) &:= \frac{1}{2} \langle\alpha, YGY\alpha\rangle- \langle1_M,\alpha\rangle\\
    \Omega &= \{\alpha \in \mathbb{R}^M : \langle y, \alpha \rangle=0 \quad \text{and} \quad 0\leq \alpha \leq C \}
\end{align*}

is a feasible set, $\pi_{\Omega}$ denotes the projection onto $\Omega$ and $\tau_k >0$ is a suitible steplength.




In [ ]:
def f(alpha, G, Y):
    return 0.5*np.dot(alpha, np.dot(np.dot(Y, np.dot(G,Y)), alpha)) - np.sum(alpha)


def gradientf(alpha, G, Y):
    return np.dot(np.dot(G,Y), alpha) - 1
    

def gradient_descent(alpha0, G, Y, tau, niter):
    alpha = alpha0

    for i in range(niter):
        d_k = projection(alpha - tau*gradientf(alpha, G, Y), G, Y) - alpha
        alpha = alpha + d_k 
        
        tau*gradientf(alpha, G, Y)
    return alpha


def projection(alpha, Y, C=1.0, tol=1e-6, max_iter=100):
    y = np.diag(Y).flatten()  
    beta = alpha.copy()
    
    
    lambda_low, lambda_high = -10, 10  
    for _ in range(max_iter):
        lambda_mid = (lambda_low + lambda_high) / 2.0
        
        
        projected_alpha = np.clip(beta + lambda_mid * y, 0, C)
        
        
        constraint_value = np.dot(y, projected_alpha)
        
        if abs(constraint_value) < tol:
            return projected_alpha  
        
        
        if constraint_value > 0:
            lambda_high = lambda_mid
        else:
            lambda_low = lambda_mid
            
    return projected_alpha 

alpha0 = np.ones(n_A)
tau = 0.01
niter = 1000
alpha = gradient_descent(alpha0, G, y, tau, niter)
print(alpha)

